In [1]:
#Libraries to use
from selenium import webdriver
import tweepy
import csv
import pandas as pd
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
from time import sleep

In [2]:
#Define token access
costumer_key = '3Fpt5yX0rRcx3HltSvOMLWcnN'
costumer_secret_key = 'WqPVb5ELVOORF2BFM059WqNJCYxXlHRKBcqaQ5Re05J6CV1ZVc'
access_token = '1393767320611442692-nLgQB8fzUWpqFRYqJcBurY8SGrnaCx'
access_token_secret = 'RsLQDfSDYHryndKlJ9CU7rI8hvE3Qlw5hpBhuoJbiqIdH'

In [3]:
#Twitter authentication 
auth = tweepy.OAuthHandler(costumer_key,costumer_secret_key)
auth.set_access_token(access_token,access_token_secret)
#Instance
api = tweepy.API(auth, wait_on_rate_limit_notify=True)

In [4]:
#Define list to use
list_tweetsID = []
list_tweets = []

### Palabras a usar:
- LGBT, LGBTQ, LGBI+
- Homosexual
- Gay
- Lesbiana

In [8]:
#LGBT word example. We run same code for each word
for tweet in tweepy.Cursor(api.search, q="LGBT", lang="es").items(3000):
    list_tweetsID.append(tweet.id)
    list_tweets.append(tweet.text)

In [9]:
len(list_tweetsID)

2651

In [10]:
dictt = dict(zip(list_tweetsID, list_tweets))

In [20]:
lista_tweets = pd.DataFrame(dictt.items(),  columns = ['TWEET_ID', 'TWEET'] )

In [13]:
#Put tweet into excel
lista_tweets.to_excel('Data/tweets_LGBT1.xlsx')

In [14]:
list_urls = []
for ide in list_tweetsID:
    list_urls.append('https://twitter.com/LGBTQ/status/' + str(ide))

In [19]:
def sacando_comentarios(cad):
    options =  webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    executable_path = r"C:\Users\Paola\AnaliticaWeb\chromedriver.exe"
    driver = webdriver.Chrome(executable_path = executable_path, chrome_options=options)
    driver.maximize_window()
    time.sleep(3)
    # Inicializamos el navegador en la url
    driver.get(cad)
    time.sleep(3)
    # conseguir altura de pagina
    last_position = driver.execute_script("return window.pageYOffset;")
    html = requests.get(cad).content

    comment_tweet=[]
    while True:
        html = driver.page_source
        soup = bs(html,'lxml')
        comments = soup.find_all('div', {'class': 'css-901oao r-1fmj7o5 r-1qd0xha r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0'})
        for comment in comments:
            comment_tweet.append(comment.text)
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            break
        else:
            last_position = curr_position     
    driver.quit()
    return comment_tweet

In [16]:
def sacando_id(link):
    sep = link.split('/')
    ide = sep[len(sep)-1]
    return(ide)

In [17]:
comentarios=[]

In [18]:
for link in list_urls:
    ide = sacando_id(link)
    tweet_comentario=sacando_comentarios(link)
    for texto in tweet_comentario:
        fila=[ide,texto]
        if fila not in comentarios:
            comentarios.append(fila)

In [64]:
comentarios_dataframe= pd.DataFrame(comentarios, columns=['TWEET_ID','COMENTARIO'])

In [66]:
comentarios_dataframe.to_excel('Data/comentarios_LGBT1.xlsx')